<a href="https://colab.research.google.com/github/waynesid/image_classification/blob/master/cat_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dog_filtered.zip',origin = _URL, extract = True)

68608000/68606236 [==============================] - 1s 0us/step


In [ ]:
zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

/root/.keras/datasets
/root/.keras/datasets/cats_and_dogs_filtered
/root/.keras/datasets/cats_and_dogs_filtered/validation
/root/.keras/datasets/cats_and_dogs_filtered/validation/dogs
/root/.keras/datasets/cats_and_dogs_filtered/validation/cats
/root/.keras/datasets/cats_and_dogs_filtered/train
/root/.keras/datasets/cats_and_dogs_filtered/train/dogs
/root/.keras/datasets/cats_and_dogs_filtered/train/cats


In [ ]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cat_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cat_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


num_cats_tr = len(os.listdir(train_cat_dir))
num_dog_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cat_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dog_tr
total_val = num_cats_val + num_dogs_val

In [ ]:
print(num_cats_tr)
print(num_dog_tr)
print(num_cats_val)
print(num_dogs_val)

1000
1000
500
500


In [ ]:
BATCH_SIZE = 32
IMG_SHAPE = 150

In [ ]:
train_image_generator = ImageDataGenerator(rescale = 1./255,
                                           horizontal_flip = True,
                                           zoom_range = 0.2,
                                           vertical_flip = True,
                                           rotation_range = 40,
                                           width_shift_range = 0.2,
                                           height_shift_range = 0.2,
                                           shear_range = 0.2,
                                           fill_mode = 'nearest')
validation_image_generator = ImageDataGenerator(rescale=1./255)


In [ ]:
train_data_gen = train_image_generator.flow_from_directory(directory = train_dir,
                                                           batch_size = BATCH_SIZE,
                                                           shuffle = True,
                                                           target_size = (IMG_SHAPE, IMG_SHAPE),
                                                           class_mode = 'binary'
                                                           )
validation_data_gen = validation_image_generator.flow_from_directory(directory = validation_dir,
                                                             batch_size = BATCH_SIZE,
                                                             shuffle = False,
                                                             target_size = (IMG_SHAPE, IMG_SHAPE),
                                                             class_mode = 'binary'
                                                             )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, (3,3), padding = 'same', input_shape = (150,150, 3), activation = 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation ='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation ='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),

                                    tf.keras.layers.Dropout(0.2),

                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation = 'relu'),
                                    tf.keras.layers.Dense(2, activation = 'softmax')

                        
                            
                            
])

In [ ]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = (['accuracy'])
              )

In [ ]:
EPOCHS = 20
history = model.fit_generator(train_data_gen,
                              steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
                              epochs = EPOCHS,
                              validation_data = validation_data_gen,
                              validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
                              )

Epoch 1/20
63/63 [==============================] - 18s 288ms/step - loss: 0.6946 - accuracy: 0.4970 - val_loss: 0.6837 - val_accuracy: 0.4990
Epoch 2/20
63/63 [==============================] - 18s 285ms/step - loss: 0.6933 - accuracy: 0.5065 - val_loss: 0.6919 - val_accuracy: 0.5890
Epoch 3/20
63/63 [==============================] - 18s 282ms/step - loss: 0.6920 - accuracy: 0.5110 - val_loss: 0.6901 - val_accuracy: 0.5550
Epoch 4/20
63/63 [==============================] - 18s 284ms/step - loss: 0.6897 - accuracy: 0.5390 - val_loss: 0.6815 - val_accuracy: 0.5760
Epoch 5/20
63/63 [==============================] - 18s 284ms/step - loss: 0.6819 - accuracy: 0.5640 - val_loss: 0.6712 - val_accuracy: 0.5920
Epoch 6/20
63/63 [==============================] - 18s 282ms/step - loss: 0.6826 - accuracy: 0.5535 - val_loss: 0.7135 - val_accuracy: 0.5260
Epoch 7/20
63/63 [==============================] - 18s 282ms/step - loss: 0.6669 - accuracy: 0.5865 - val_loss: 0.6788 - val_accuracy: 0.5720